In [1]:
import os
import json
import traceback
import pandas as pd

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
KEY = os.getenv('GOOGLE_API_KEY')


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\ASUS\mcqgeneration_google_gemini\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [11]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [37]:
TEMPLATE = """
Text : {text}
You are an expert MCQ maker. Given the text above, it is you job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}
"""   

In [38]:
RESPONSE_JSON = {
    "1":{
        "mcq": "mutliple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct choice here"
        },
    "2":{
        "mcq": "mutliple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct choice here"
        },
    "3":{
        "mcq": "mutliple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct choice here"
        },
    }

In [39]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE,
    )

In [40]:
quiz_chain = LLMChain( llm = llm, prompt = quiz_generation_prompt, output_key="quiz", verbose=True)

In [41]:
TEMPLATE2 = """
You are a expert english grammarian and writer. Given the Multiple Choice Questions for {subject} students.\
You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.\
if the quiz is not at per the with the cognitive and analytical ablities of the students.\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students cognitive and analytical abilities.\
Quiz_MCQs: 
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [42]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2,
    )

In [43]:
review_chain = LLMChain( llm = llm, prompt = quiz_evaluation_prompt, output_key="review", verbose=True)

In [44]:
generate_evaluate_chain = SequentialChain( chains = [quiz_chain, review_chain], input_variables= ["text", "number", "tone", "subject", "response_json"] , output_variables=["quiz", "review"] ,verbose=True)

In [45]:
file_path = r"C:\Users\ASUS\mcqgeneration_google_gemini\data.txt"

In [46]:
with open(file_path , 'r') as file:
    TEXT = file.read()
TEXT

'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contrib

In [47]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "mutliple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct choice here"}, "2": {"mcq": "mutliple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct choice here"}, "3": {"mcq": "mutliple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct choice here"}}'

In [48]:
NUMBER = 5
SUBJECT  = "Machine Learning"
TONE = "Simple"

In [49]:
response = generate_evaluate_chain({
    "text": TEXT,
    "number": NUMBER,
    "tone": TONE,
    "subject": SUBJECT,
    "response_json": json.dumps(RESPONSE_JSON)
})
print(response["quiz"].strip()) 



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons use

In [53]:
text = response.get("quiz")
json_start_index = text.find('{')
json_str = text[json_start_index:]
json_data = json.loads(json_str)

In [54]:
json_data

{'1': {'mcq': 'Who is known as the pioneer in the field of computer gaming and artificial intelligence and coined the term machine learning in 1959?',
  'options': {'a': 'Donald Hebb',
   'b': 'Arthur Samuel',
   'c': 'Walter Pitts',
   'd': 'Tom M. Mitchell'},
  'correct': 'b'},
 '2': {'mcq': 'What was the name of the experimental learning machine developed by Raytheon Company?',
  'options': {'a': 'Cybertron',
   'b': 'Learning Machine',
   'c': 'Pattern Recognizer',
   'd': 'Artificial Neural Network'},
  'correct': 'a'},
 '3': {'mcq': "Which psychologist's work on neural structure influenced the development of machine learning algorithms?",
  'options': {'a': 'Walter Pitts',
   'b': 'Warren McCulloch',
   'c': 'Donald Hebb',
   'd': 'Tom M. Mitchell'},
  'correct': 'c'},
 '4': {'mcq': 'What is a widely quoted definition of machine learning given by Tom M. Mitchell?',
  'options': {'a': 'Machine learning is the ability of a computer to learn without being explicitly programmed.',
  

In [57]:
quiz_table_data = []
for key, value in json_data.items():
    mcq = value["mcq"]
    options =" | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
        ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ" : mcq, "Options" : options, "Correct" : correct})

In [58]:
quiz_table_data

[{'MCQ': 'Who is known as the pioneer in the field of computer gaming and artificial intelligence and coined the term machine learning in 1959?',
  'Options': 'a : Donald Hebb | b : Arthur Samuel | c : Walter Pitts | d : Tom M. Mitchell',
  'Correct': 'b'},
 {'MCQ': 'What was the name of the experimental learning machine developed by Raytheon Company?',
  'Options': 'a : Cybertron | b : Learning Machine | c : Pattern Recognizer | d : Artificial Neural Network',
  'Correct': 'a'},
 {'MCQ': "Which psychologist's work on neural structure influenced the development of machine learning algorithms?",
  'Options': 'a : Walter Pitts | b : Warren McCulloch | c : Donald Hebb | d : Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': 'What is a widely quoted definition of machine learning given by Tom M. Mitchell?',
  'Options': 'a : Machine learning is the ability of a computer to learn without being explicitly programmed. | b : Machine learning is the study of algorithms that can improve their perform

In [60]:
quiz = pd.DataFrame(quiz_table_data)

In [61]:
quiz.to_csv("machine_learning_quiz.csv", index=False)